# Install Requirements

In [ ]:
import sys
import subprocess

def install_packages():
    """Automatically install required packages"""
    packages = [
        'boto3>=1.26.0',
        'requests>=2.26.0',
        'requests-aws4auth>=1.3.0',
        'pillow==11.2.1',
        # 'ipython==8.30.0'
    ]
    
    print("📦 Installing required packages...")
    for package in packages:
        try:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
            print(f"✅ {package} installed successfully")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install {package}: {e}")
    
    print("\n🎉 Package installation complete! Run the next cell.")

install_packages()

In [ ]:
import json
import time
from bedrock_text import BedrockText
from bedrock_image import BedrockAmazonImage, NovaImageSize
from bedrock_video import BedrockAmazonVideo, VideoStatus
from bedrock_model import BedrockModel
from image_utils import (
    get_image_bytes_from_file,
    bytes_to_base64,
    display_image,
    display_image_bytes,
    display_video_bytes,
)

## 0. Declare required models, classes, variables, and data

In [ ]:
AWS_REGION = "us-east-1"
S3_VIDEO_BUCKET = "<your-bucket-name>" # Replace with you S3 bucket name
    
# (Required) face image path
SAMPLE_IMAGE_PATH = "./img/sample_face.png"
SAMPLE_IMAGE_FORMAT = "png"

if S3_VIDEO_BUCKET == "<your-bucket-name>":
    print("⚠️ WARNING: Please update the 'S3_VIDEO_BUCKET' with your actual S3 bucket name.")

In [ ]:
# LLM model
bedrock_llm = BedrockText(modelId=BedrockModel.NOVA_LITE_CR, region=AWS_REGION)

# Image model
bedrock_image = BedrockAmazonImage(region=AWS_REGION, modelId=BedrockModel.NOVA_CANVAS)

# Video model
bedrock_video = BedrockAmazonVideo(
    region=AWS_REGION,
    modelId=BedrockModel.NOVA_REEL,
    bucket_name=S3_VIDEO_BUCKET
)

In [ ]:
# sample image
img_bytes = get_image_bytes_from_file(file_path=SAMPLE_IMAGE_PATH, format=SAMPLE_IMAGE_FORMAT)
display_image_bytes(img_bytes)

In [ ]:
def extract_json(str):
    try:
        str = str.replace('```json', '').replace('```', '')
        return json.loads(str)
    except Exception as e:
        print(f"Error: {str}")
        return {}

## 1. Extract character personas and episodes from image (Nova Lite)

In [ ]:
PROMPT = f"""You are an empathetic AI storyteller who infers mood and personality from a person's face to create a lighthearted story based on everyday life.

## Instruction

1. Analyze the person's gender, age range, impression, and personality based on the image.
2. Define the persona of the individual, reflecting their character and possible profession.
3. Based on the inferred gender, age, personality, and profession, write a playful or funny short episode. Make it trendy and cheerful, and limit it to a maximum of 2 sentences.

## Output Format

Please provide your description in the following format, without any additional explanation:
{{
    "persona": "1–2 sentence summary describing gender, age range, impression, personality, and profession",
    "episode": "A playful and trendy short episode, maximum two sentences"
}}
"""

res = bedrock_llm.converse_output(text=PROMPT, image=img_bytes, format=SAMPLE_IMAGE_FORMAT)
j = extract_json(res)
persona = j.get('persona', '')
episode = j.get('episode', '')

print(persona)
print(episode)

## 2. Generate a Image Prompt (Nova Lite)

In [ ]:
PROMPT = f"""You are a professional prompt engineer specializing in image generation models.
Based on the given image, create a prompt that preserves the subject's facial features as accurately as possible, while vividly visualizing the scene described below.
Emphasize the context of the episode clearly, but ensure the character's appearance remains consistent.

- persona: {persona}
- episode: {episode}

## Output Format

Please provide your description in the following format, without any additional explanation:
{{
    "prompt": "A single English sentence for image generation, clearly describing the character’s features and the scene."
}}
"""

res = bedrock_llm.converse_output(text=PROMPT)
prompt = json.loads(res).get('prompt', '')
print(prompt)

## 3. Generate a character using an image and generated prompts (Nova Canvas)

In [ ]:
image_base64 = bytes_to_base64(img_bytes)
width, height = NovaImageSize.SIZE_1280x720.value
cfgScale = 9.0
similarity = 0.9
seed = 512

body = json.dumps({
    "taskType": "IMAGE_VARIATION",
    "imageVariationParams": {
        "images": [image_base64],
        "similarityStrength": similarity,
        "text": f"{prompt}, cartoon style, colorful digital illustration",
        "negativeText": "ugly, deformed, low quality, blurry, text"
    },
    "imageGenerationConfig": {
        "numberOfImages": 1,
        "width": width,
        "height": height,
        "cfgScale": cfgScale,
        "seed": seed
    }
})

img = bedrock_image.generate_image(body)
display_image(img)

## 4. Generate a video prompt (Nova Lite)

In [ ]:
PROMPT = f"""You are a prompt engineer specializing in crafting prompts for image-to-video generation.
Refer to the given image and image description to write a dynamic, high-quality video generation prompt in English

## Image Description
{prompt}

## Instruction
Video generation models work best with descriptive prompts rather than command-based instructions. When crafting prompts, focus on what you want to see rather than telling the model what to do. Think of writing a detailed caption or scene description like you’re explaining a video that already exists. For example, describing elements like the main subjects, what they’re doing, the setting around them, how the scene is lit, the overall artistic style, and any camera movements can help create more accurate results. The key is to paint a complete picture through description rather than giving step-by-step directions. This means instead of saying “create a dramatic scene,” you could describe “a stormy beach at sunset with crashing waves and dark clouds, filmed with a slow aerial shot moving over the coastline.” The more specific and descriptive you can be about the visual elements you want, the better the output will be. You might want to include details about the subject, action, environment, lighting, style, and camera motion.

When writing a video generation prompt for Nova Reel, be mindful of the following requirements and best practices:

Prompts must be no longer than 500 characters.
For best results, place camera movement descriptions at the start or end of your prompt.
Specify what you want to include rather than what to exclude. For example, instead of “fruit basket with no bananas,” say “fruit basket with apples, oranges, and pears.”
When describing camera movements in your video prompts, be specific about the type of motion you want—whether it’s a smooth dolly shot (moving forward/backward), a pan (sweeping left/right), or a tilt (moving up/down). For more dynamic effects, you can request aerial shots, orbit movements, or specialized techniques like dolly zooms. You can also specify the speed of the movement

## Example
- Track right shot of single red balloon floating through empty subway tunnel. Balloon glows from within, casting soft red light on concrete walls. Cinematic 4k, moody lighting
- Dolly in shot of peaceful deer drinking from forest stream. Sunlight filtering through and bokeh of other deer and forest plants. 4k cinematic.
- Orbit shot of crystal light bulb centered on polished marble surface, gentle floating gears spinning inside with golden glow. Premium lighting. 4k cinematic.
- Pedestal down in a pan in modern kitchen penne pasta with heavy cream white sauce on top, mushrooms and garlic, steam coming out.
- Orbit shot of premium over-ear headphones on a reflective surface. Dramatic side lighting accentuates the curves and edges, casting subtle shadows that highlight the product’s premium build quality.

## Output format

Please provide your description in the following format, without any additional explanation:
{{
    "video_prompt": "Video generation prompt in English"
}}
"""

res = bedrock_llm.converse_output(text=PROMPT)
video_prompt = json.loads(res).get('video_prompt', '')
print(video_prompt)

## 5. Generate a single-shot video from image and text prompt

In [ ]:
invocation_arn = bedrock_video.generate_video(
    text=video_prompt,
    image=img[0],
    imageFormat="png",
    durationSeconds=6,
    fps=24,
    dimension="1280x720"
)

status, s3Uri, invocation = bedrock_video.query_job(invocation_arn)

while status == VideoStatus.IN_PROGRESS:
    time.sleep(30)
    status, s3Uri, invocation = bedrock_video.query_job(invocation_arn)

video = bedrock_video.get_video(invocation_arn=invocation_arn)
display_video_bytes(video)

## 6. Generate a multi-shot video from text prompt

In [ ]:
invocation_arn = bedrock_video.generate_multishot_video(
    text=video_prompt,
    durationSeconds=24,
    fps=24,
    dimension="1280x720"
)

status, s3Uri, invocation = bedrock_video.query_job(invocation_arn)

while status == VideoStatus.IN_PROGRESS:
    time.sleep(30)
    status, s3Uri, invocation = bedrock_video.query_job(invocation_arn)

video = bedrock_video.get_video(invocation_arn=invocation_arn)
display_video_bytes(video)